In [14]:
import pandas as pd

class MarkDownTable:
    def __init__(self, num_rows, num_cols):
        # Initialize the table with empty strings
        self.table = pd.DataFrame([""] * num_cols).T
        
        for _ in range(num_rows - 1):
            self.table = pd.concat([self.table, pd.DataFrame([""] * num_cols).T], ignore_index=True)
            
        self.num_rows = num_rows
        self.num_cols = num_cols
            
    def _repr_(self):
        return self.table._repr_()
    
    def _repr_html_(self):
        return self.table._repr_html_()

    def add_to_row(self, row_index, lambda_func):
        # create the new row data to be inserted
        new_row_data = [lambda_func(col) for col 
                        in range(self.num_cols)]
        
        # Add a new row if row_index is -1 or does not exist
        if row_index == -1 or row_index >= len(self.table):
            self.table = pd.concat([self.table, pd.DataFrame(new_row_data).T], ignore_index=True)
            self.num_rows += 1 # update the number of rows
            
        else: # modify the row with new data
            self.table.loc[row_index] = new_row_data

    def add_to_col(self, col_index, lambda_func):
        # create the new col data to be inserted
        new_row_data = [lambda_func(row) for row 
                        in range(self.num_rows)]
        
        # Add a new column if col_index does not exist
        if col_index >= len(self.table.columns):
            self.table[col_index] = new_row_data
            
        else: # modify the column with new data
            self.table[col_index] = self.table[col_index].apply(lambda_func)

    def format_all(self, **kwargs):
        # Apply styling to all elements in the table
        self.table_styling = self.table.style.set_properties(**kwargs)

    def format_row(self, row_index, **kwargs):
        # Apply styling to a specific row
        self.table_styling = self.table.style.apply(lambda x: kwargs, subset=pd.IndexSlice[row_index, :])

    def format_col(self, col_index, **kwargs):
        # Apply styling to a specific column
        self.table_styling = self.table.style.apply(lambda x: kwargs, axis=0, subset=pd.IndexSlice[:, col_index])

    def to_pandas(self):
        # Return the table as a pandas DataFrame
        return self.table

    def to_markdown(self):
        # Convert the table to markdown format
        return self.table.to_markdown(index=False)


def pandas_to_markdown(dataframe, output='markdown'):
    if output == 'markdown':
        return dataframe.to_markdown(index=False)
    elif output == MarkDownTable:
        table = MarkDownTable(dataframe.shape[0], dataframe.shape[1])
        for row_index in range(dataframe.shape[0]):
            for col_index in range(dataframe.shape[1]):
                table.table.iloc[row_index, col_index] = dataframe.iloc[row_index, col_index]
                
        return table # instance of pd.DataFrame or MarkDownTable

# Example Usage
table = MarkDownTable(2, 3)
table.add_to_row(0, lambda col: f"Row0-Col{col}")
table.add_to_col(2, lambda row: f"Col2-Row{row}")
print(table.to_markdown())

# Convert a pandas DataFrame to markdown
df = pd.DataFrame({'A': [1, 2], 'B': [3, 4]})

| 0         | 1         | 2                 |
|:----------|:----------|:------------------|
| Row0-Col0 | Row0-Col1 | Col2-RowRow0-Col2 |
|           |           | Col2-Row          |


In [15]:
table

,0,1,2
0,Row0-Col0,Row0-Col1,Col2-RowRow0-Col2
1,,,Col2-Row
